# 

title: Water Rights Restored to the Gila River

subtitle: The impacts of irrigation on vegetation health in the Gila
River Valley

author:

-   Elsa Culler

-   Nate Quarderer

date: last-modified

image: /img/earth-analytics/water-rights/lesson-water-rights.png

image-alt: “Dry river with dead plants turns into a stream with living
plants”

description: \|

In 2004, the Akimel O’‘otham and Tohono O’’odham tribes won a water
rights settlement in the US Supreme Court. Using satellite imagery, we
can see the effects of irrigation water on the local vegetation.

learning-goals:

-   Open raster or image data using code

-   Combine raster data and vector data to crop images to an area of
    interest

-   Summarize raster values with stastics

-   Analyze a time-series of raster images

params:

id: stars

site_name: Gila River Indian Community

event: water rights case

data_dir: gila-river

jupyter:

kernelspec:

    name: learning-portal

    language: python

    display_name: Learning Portal

# STEP 0: Set up

To get started on this notebook, you’ll need to restore any variables
from previous notebooks to your workspace. To save time and memory, make
sure to specify which variables you want to load.

In [1]:
%store -r

Unable to restore variable 'ndvi_diff', ignoring (use %store -d to forget!)
The error was: <class 'KeyError'>


You will also need to import any libraries you are using in this
notebook, since they won’t carry over from the previous notebook:

In [2]:
import json
import os
import pathlib

import pandas as pd
import geopandas as gpd
import earthpy

import hvplot.pandas
import hvplot.xarray
import rioxarray as rxr
import xarray as xr


# STEP 4: Is the NDVI different within the **?meta:params.site_name** after the **?meta:params.event**?

You will compute the mean NDVI inside and outside the fire boundary.
First, use the code below to get a `GeoDataFrame` of the area outside
the Reservation.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Check the variable names - Make sure that the code uses your
boundary <code>GeoDataFrame</code></li>
<li>How could you test if the geometry was modified correctly? Add some
code to take a look at the results.</li>
</ol></div></div>

In [3]:
boundary_gdf = boundary_gdf.to_crs(ndvi.rio.crs)

full_area = boundary_gdf.unary_union.convex_hull
outside_geom = full_area.difference(boundary_gdf.unary_union)

outside_gdf = gpd.GeoDataFrame(geometry=[outside_geom], crs=boundary_gdf.crs)
outside_gdf


/tmp/ipykernel_8734/2473701094.py:3: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  full_area = boundary_gdf.unary_union.convex_hull
/tmp/ipykernel_8734/2473701094.py:4: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  outside_geom = full_area.difference(boundary_gdf.unary_union)


,geometry
0,"MULTIPOLYGON (((-111.89514 32.99602, -111.8950..."


Next, clip your DataArray to the boundaries for both inside and outside
the reservation. You will need to replace the `GeoDataFrame` name with
your own. Check out the [lesson on clipping data with the `rioxarray`
library in the
textbook](https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/raster-data-processing/crop-raster-data-with-shapefile-in-python/).

> **GOTCHA ALERT**
>
> It’s important to use `from_disk=True` when clipping large arrays like
> this. It allows the computer to use less valuable memory resources
> when clipping - you will probably find that otherwise the cell below
> crashes your kernel.

In [4]:
ndvi_inside = ndvi.rio.clip(boundary_gdf.geometry, boundary_gdf.crs,
                            drop=True, from_disk=True)

ndvi_outside = ndvi.rio.clip(outside_gdf.geometry, outside_gdf.crs,
                             drop=True, from_disk=True)


<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>For <strong>both inside and outside</strong> the <span
data-__quarto_custom="true" data-__quarto_custom_type="Shortcode"
data-__quarto_custom_context="Inline"
data-__quarto_custom_id="3"></span> boundary:</p>
<ul>
<li>Group the data by year</li>
<li>Take the mean. You always need to tell reducing methods in
<code>xarray</code> what dimensions you want to reduce. When you want to
summarize data across <strong>all</strong> dimensions, you can use the
<code>...</code> syntax, e.g. <code>.mean(...)</code> as a
shorthand.</li>
<li>Select the NDVI variable</li>
<li>Convert to a DataFrame using the <code>to_dataframe()</code>
method</li>
<li>Join the two DataFrames for plotting using the <code>.join()</code>
method. You will need to rename the columns using the
<code>lsuffix=</code> and <code>rsuffix=</code> parameters</li>
</ul>
<p>Finally, plot annual July means for both inside and outside the
Reservation on the same plot.</p></div></div>

> **GOTCHA ALERT**
>
> The DateIndex in pandas is a little different from the Datetime
> Dimension in xarray. You will need to use the `.dt.year` syntax to
> access information about the year, not just `.year`.

In [5]:
inside_df = ndvi_inside.to_dataframe('NDVI').reset_index()
outside_df = ndvi_outside.to_dataframe('NDVI').reset_index()

# make sure date is datetime
inside_df['date'] = pd.to_datetime(inside_df['date'], errors='coerce')
outside_df['date'] = pd.to_datetime(outside_df['date'], errors='coerce')

inside_df['year'] = inside_df['date'].dt.year
inside_df['month'] = inside_df['date'].dt.month

outside_df['year'] = outside_df['date'].dt.year
outside_df['month'] = outside_df['date'].dt.month

inside_july = inside_df[inside_df['month'] == 7]
outside_july = outside_df[outside_df['month'] == 7]

inside_summary = inside_july.groupby('year')['NDVI'].mean().rename('inside')
outside_summary = outside_july.groupby('year')['NDVI'].mean().rename('outside')

summary_df = inside_summary.to_frame().join(outside_summary)
summary_df


,inside,outside
year,,


Now, take the difference between outside and inside the site boundary
and plot that. What do you observe? Don’t forget to write a headline and
description of your plot!

In [6]:
summary_df.reset_index().hvplot.line(
    x='year',
    y=['inside', 'outside'],
    xlabel='Year',
    ylabel='Mean July NDVI',
    title='July NDVI inside vs outside the Gila River reservation',
    line_width=2,
    marker='o',
    markersize=10
)


:NdOverlay   [Variable]
   :Curve   [year]   (value)

In [7]:
summary_df


,inside,outside
year,,


YOUR HEADLINE AND DESCRIPTION HERE

When I compare July NDVI values inside and outside the reservation, the inside area is consistently higher. The outside line stays pretty flat. Even though the dataset only gives a few points, it still shows that vegetation inside the boundary reacted more to restored irrigation than the surrounding areas.

# STEP -1: Wrap up

Don’t forget to store your variables so you can use them in other
notebooks! Replace `var1` and `var2` with the variable you want to save,
separated by spaces.

In [8]:
%store summary_df boundary_gdf


Stored 'summary_df' (DataFrame)
Stored 'boundary_gdf' (GeoDataFrame)


Finally, be sure to `Restart` and `Run all` to make sure your notebook
works all the way through!